## CSV

### Read

In [19]:
df = spark.read.format("csv") \
  .option("header", "true") \
  .option("mode", "FAILFAST") \
  .load("/work/data/flight-data/csv/2010-summary.csv")
df.show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
+-----------------+-------------------+-----+
only showing top 3 rows



### Write

In [20]:
df.write.format("csv") \
    .mode("overwrite") \
    .option("sep", "\t") \
    .save("/tmp/my-tsv-file.tsv")

## JSON

In [21]:
### Read

In [22]:
df = spark.read.format("json").option("mode", "FAILFAST") \
  .option("inferSchema", "true") \
  .load("/work/data/flight-data/json/2010-summary.json")
df.show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
+-----------------+-------------------+-----+
only showing top 3 rows



### Write

In [23]:
df.write.format("json").mode("overwrite").save("/tmp/my-json-file.json")

## Parquet

### Read

In [24]:
# optionally with format('parquet')
df = spark.read.load("/work//data/flight-data/parquet/2010-summary.parquet")
df.show(5)


+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



### Write

In [25]:
df.write.mode("overwrite").save("/tmp/my-parquet-file.parquet")

In [ ]:
SQL

In [1]:
path = "/work/data/flight-data/jdbc/my-sqlite.db"
url = "jdbc:sqlite:" + path
driver = "org.sqlite.JDBC"
tablename = "flight_info"

dbDataFrame = spark.read.format("jdbc").option("url", url).option("dbtable", tablename).option("driver",  driver).load()
dbDataFrame.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows

